In [5]:
import pip
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,BatchNormalization,Softmax,InputLayer
import numpy as np
import splitfolders
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile,Image

## Model Structure

In [24]:
classifier = Sequential()



classifier.add(Conv2D(filters = 8 ,strides=(1,1), kernel_size=(3,3),activation='relu',padding='same',input_shape=(32,32,1)))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters = 16, kernel_size=(3,3),padding='same',activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(filters = 32, kernel_size=(3,3),padding='same',activation='relu'))
#classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Flatten())
classifier.add(Dense(120))
classifier.add(Dense(60,'Softmax'))


classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])



In [6]:
splitfolders.fixed("C:\\Users\Saunak\Desktop\Image_splitting_test_run\BanglaLekha-Isolated\Images", output="output",
    seed=1337, fixed=(1500, 437), oversample=False, group_prefix=None, move=False)

Copying files: 166105 files [20:54, 132.37 files/s]


In [25]:
classifier.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 32, 32, 8)         80        
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 16, 16, 8)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 16, 16, 16)        1168      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 8, 8, 32)          4640      
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 4, 4, 32)        

In [26]:
train_folder = "output/train1/"
test_folder = "output/val"

In [28]:
ImageFile.LOAD_TRUNCATED_IMAGES = True


train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = .2, rotation_range = 25)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_folder, target_size = (32, 32), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")
test_set = test_datagen.flow_from_directory(test_folder, target_size = (32, 32), 
                                                 batch_size = 1, class_mode = 'categorical',color_mode="grayscale")


Found 90000 images belonging to 60 classes.
Found 26220 images belonging to 60 classes.


In [30]:

classifier.fit(training_set ,steps_per_epoch = 90000, epochs = 10,
                validation_data = test_set, validation_steps = 20000)

Epoch 1/10
90000/90000 [==============================] - 385s 4ms/step - loss: 0.8027 - accuracy: 0.7723 - val_loss: 0.7043 - val_accuracy: 0.8061
Epoch 2/10
90000/90000 [==============================] - 282s 3ms/step - loss: 0.7759 - accuracy: 0.7837 - val_loss: 0.7698 - val_accuracy: 0.7822
Epoch 3/10
90000/90000 [==============================] - 286s 3ms/step - loss: 0.7638 - accuracy: 0.7864 - val_loss: 0.6981 - val_accuracy: 0.8064
Epoch 4/10
90000/90000 [==============================] - 277s 3ms/step - loss: 0.7526 - accuracy: 0.7881 - val_loss: 0.7104 - val_accuracy: 0.8103
Epoch 5/10
90000/90000 [==============================] - 268s 3ms/step - loss: 0.7505 - accuracy: 0.7881 - val_loss: 0.8104 - val_accuracy: 0.7868
Epoch 6/10
90000/90000 [==============================] - 270s 3ms/step - loss: 0.7443 - accuracy: 0.7899 - val_loss: 0.7451 - val_accuracy: 0.8098
Epoch 7/10
90000/90000 [==============================] - 272s 3ms/step - loss: 0.7412 - accuracy: 0.7918 - val_

In [ ]:
classifier_json = classifier.to_json()

with open("CNN_Bangla_Paper_Model.json", "w") as json_file:
    json_file.write(classifier_json)
    
classifier.save_weights("CNN_Bangla_Paper_Model.h5")
print('Saved model to disk')